In [66]:
import pandas
import sqlite3
import numpy as np

In [67]:
connection = sqlite3.connect('nearby_stars.db')
c = connection.cursor()

## Simbad TAP query information

The <code>simbad_FGK_nobinaries.csv</code> file was created using the ADQL query in the next cell through the Simbad TAP (Table Access Protocol") service. I used TOPCAT to query Simbad and then saved the resulting table as a .csv file, but the TAP service can also be accessed here: https://simbad.u-strasbg.fr/simbad/sim-tap

Information on available Simbad tables is here: http://simbad.u-strasbg.fr/simbad/tap/tapsearch.html

**Simbad TAP query**
<code>
SELECT sub.main_id, sub.ra, sub.dec,  sub_hd.id as hd, sub_gj.id as gj, sub_hip.id as hip, sub_wds.id as wds, sub.sp_type, sub.sp_bibcode, 
sub.plx_value as plx_value, sub.plx_err, sub.plx_bibcode, 
sub_v.flux as V, sub_v.flux_err as V_err, sub_v.bibcode as V_bibcode, sub_g.flux as G, sub_g.flux_err as G_err, sub_g.bibcode as G_bibcode
FROM (
SELECT basic.main_id, basic.oid, basic.ra, basic.dec, (alltypes.otypes || '|' || otypedef.otype_shortname) AS otypes_all, basic.sp_type, basic.sp_bibcode, 
basic.plx_value, basic.plx_err, basic.plx_bibcode, allfluxes.V, allfluxes.G
FROM alltypes
    INNER JOIN basic ON basic.oid = alltypes.oidref
    LEFT OUTER JOIN otypedef ON basic.otype = otypedef.otype
    LEFT OUTER JOIN h_link ON h_link.parent = alltypes.oidref
    LEFT OUTER JOIN basic AS basic_child ON h_link.child = basic_child.oid
    LEFT OUTER JOIN alltypes AS alltypes_child ON h_link.child = alltypes_child.oidref
    LEFT OUTER JOIN allfluxes ON allfluxes.oidref = basic.oid
WHERE basic.plx_value > 0
AND 1/(basic.plx_value\*1e-3) &#60; 20
AND (basic.sp_type LIKE 'F%' OR basic.sp_type LIKE 'G%' OR basic.sp_type LIKE 'K%')
AND ((basic.sp_type NOT LIKE '%II%') 
    OR (basic.sp_type NOT LIKE '%I%' AND basic.sp_type NOT LIKE '%V%'))
AND ((alltypes.otypes || '|' || otypedef.otype_shortname LIKE '%\*\*%' 
    AND alltypes.otypes || '|' || otypedef.otype_shortname NOT LIKE '%SB%' 
    AND alltypes_child.otypes LIKE '%Pl%') 
    OR (alltypes.otypes || '|' || otypedef.otype_shortname NOT LIKE '%\*\*%' 
    AND alltypes.otypes || '|' || otypedef.otype_shortname NOT LIKE '%SB\*%' 
    AND alltypes.otypes || '|' || otypedef.otype_shortname NOT LIKE '%EB\*%'))
AND (basic.main_id NOT LIKE '% A' AND basic.main_id NOT LIKE '% B' AND basic.main_id NOT LIKE '% C' AND basic.main_id NOT LIKE '% D')
GROUP BY basic.main_id,basic.oid, basic.ra, basic.dec, otypes_all, basic.sp_type, basic.plx_value, basic.plx_err, allfluxes.V, allfluxes.G
) AS sub
LEFT OUTER JOIN (
    SELECT basic.oid, STRING_AGG(ident.id,',') as id
    FROM basic
        INNER JOIN ident ON basic.oid = ident.oidref
    WHERE (ident.id LIKE 'WDS%')
    AND ident.id NOT LIKE '%.0'
    GROUP BY basic.oid
    ) AS sub_wds ON sub.oid = sub_wds.oid
LEFT OUTER JOIN (
    SELECT basic.oid, STRING_AGG(ident.id,',') as id
    FROM basic
        INNER JOIN ident ON basic.oid = ident.oidref
    WHERE (ident.id LIKE 'HD%')
    AND ident.id NOT LIKE '%.0'
    GROUP BY basic.oid
    ) AS sub_hd ON sub.oid = sub_hd.oid
LEFT OUTER JOIN (
    SELECT basic.oid, STRING_AGG(ident.id,',') as id
    FROM basic
        INNER JOIN ident ON basic.oid = ident.oidref
    WHERE (ident.id LIKE 'GJ%')
    AND ident.id NOT LIKE '%.0'
    GROUP BY basic.oid
    ) AS sub_gj ON sub.oid = sub_gj.oid
LEFT OUTER JOIN (
    SELECT basic.oid, STRING_AGG(ident.id,',') as id
    FROM basic
        INNER JOIN ident ON basic.oid = ident.oidref
    WHERE (ident.id LIKE 'HIP%')
    AND ident.id NOT LIKE '%.0'
    GROUP BY basic.oid
    ) AS sub_hip ON sub.oid = sub_hip.oid
LEFT OUTER JOIN (
    SELECT basic.oid, flux.filter, flux.flux, flux.flux_err, flux.bibcode
    FROM basic INNER JOIN flux ON flux.oidref = basic.oid 
    WHERE flux.filter = 'V') AS sub_v ON sub.oid = sub_v.oid
LEFT OUTER JOIN (
    SELECT basic.oid, flux.filter, flux.flux, flux.flux_err, flux.bibcode
    FROM basic INNER JOIN flux ON flux.oidref = basic.oid 
    WHERE flux.filter = 'G') AS sub_g ON sub.oid = sub_g.oid
GROUP BY sub.main_id, sub.ra, sub.dec,  hd, gj, hip, wds, sub.sp_type, sub.sp_bibcode,  
plx_value, sub.plx_err, sub.plx_bibcode,
 V, V_err, V_bibcode, G, G_err, G_bibcode
HAVING ((sub_hd.id NOT LIKE '%A%' AND sub_hd.id NOT LIKE '%B%') OR sub_hd.id IS NULL)
    AND ((sub_gj.id NOT LIKE '%A%' AND sub_gj.id NOT LIKE '%B%') OR sub_gj.id IS NULL)
ORDER BY plx_value DESC
</code>

## Add TAP query saved in .csv file to database

In [16]:
dat = pandas.read_csv('simbad_FGK_nobinaries.csv')

In [26]:
c.execute('DROP TABLE IF EXISTS stars_FGK_nobinaries')

In [17]:
dat.to_sql('stars_FGK_nobinaries', connection, index_label='sid')

In [6]:
c.execute('PRAGMA table_info(stars_FGK_nobinaries);')
c.fetchall()

[(0, u'sid', u'INTEGER', 0, None, 1),
 (1, u'main_id', u'TEXT', 0, None, 0),
 (2, u'ra', u'TEXT', 0, None, 0),
 (3, u'dec', u'TEXT', 0, None, 0),
 (4, u'hd', u'TEXT', 0, None, 0),
 (5, u'gj', u'TEXT', 0, None, 0),
 (6, u'hip', u'TEXT', 0, None, 0),
 (7, u'wds', u'TEXT', 0, None, 0),
 (8, u'sp_type', u'TEXT', 0, None, 0),
 (9, u'sp_bibcode', u'TEXT', 0, None, 0),
 (10, u'plx_value', u'REAL', 0, None, 0),
 (11, u'plx_err', u'REAL', 0, None, 0),
 (12, u'plx_bibcode', u'TEXT', 0, None, 0),
 (13, u'v', u'REAL', 0, None, 0),
 (14, u'v_err', u'REAL', 0, None, 0),
 (15, u'v_bibcode', u'TEXT', 0, None, 0),
 (16, u'G', u'REAL', 0, None, 0),
 (17, u'G_err', u'REAL', 0, None, 0),
 (18, u'G_bibcode', u'TEXT', 0, None, 0)]

In [19]:
connection.commit()

## Make sid ("star id") column a Primary Key

Create a temporary backup table.

In [28]:
c.execute('DROP TABLE IF EXISTS stars_backup')

In [29]:
cmd = '''
CREATE TEMPORARY TABLE IF NOT EXISTS stars_backup(sid, main_id, ra, dec, hd, gj, hip, wds, sp_type, 
sp_bibcode, plx_value, plx_err, plx_bibcode, v, v_err,
       v_bibcode, g, g_err, g_bibcode);
'''

c.execute(cmd)

In [30]:
cmd = '''
INSERT INTO stars_backup SELECT * FROM stars_FGK_nobinaries;
'''

c.execute(cmd)

In [31]:
cmd = '''
DROP TABLE stars_FGK_nobinaries
'''
c.execute(cmd)

In [32]:
cmd = '''
CREATE TABLE stars_FGK_nobinaries (
sid INTEGER PRIMARY KEY AUTOINCREMENT,
main_id TEXT,
ra REAL,
dec REAL,
hd TEXT,
gj TEXT,
hip TEXT,
wds TEXT,
sp_type TEXT,
sp_bibcode TEXT,
plx_value REAL,
plx_err REAL,
plx_bibcode TEXT,
v REAL,
v_err REAL,
v_bibcode TEXT,
G REAL,
G_err REAL,
G_bibcode TEXT
)
'''
c.execute(cmd)

In [33]:
cmd = '''
INSERT INTO stars_FGK_nobinaries SELECT * FROM stars_backup
'''
c.execute(cmd)

In [34]:
connection.commit()

In [35]:
c.execute("SELECT name FROM sqlite_master WHERE type='table';")
c.fetchall()

[(u'sqlite_sequence',),
 (u'stars_simbad',),
 (u'mh08_agetable',),
 (u'gaspar13_agetable',),
 (u'ramirez13_agetable',),
 (u'ramirez12_agetable',),
 (u'isaacson10_agetable',),
 (u'tsantaki13_agetable',),
 (u'casagrande11_agetable',),
 (u'chen01_agetable',),
 (u'david15_agetable',),
 (u'yee17_agetable',),
 (u'ages',),
 (u'luck17_agetable',),
 (u'gray06_rhktable',),
 (u'gray03_rhktable',),
 (u'jenkins06_rhktable',),
 (u'jenkins11_rhktable',),
 (u'stars_M0to4_nobinaries',),
 (u'stars_FGK_nobinaries',)]

## Add in M0 - M4 stars within 20 pc

In [9]:
cmd = '''
DROP TABLE stars_M0to4_nobinaries
'''
c.execute(cmd)

In [10]:
dat = pandas.read_csv('simbad_M0to4_nobinaries.csv')

In [11]:
dat.to_sql('stars_M0to4_nobinaries', connection)

In [16]:
cmd = '''INSERT INTO stars_FGK_nobinaries (main_id, ra, dec, hd, gj,hip,wds,sp_type,sp_bibcode,plx_value,plx_err,plx_bibcode,v,v_err,v_bibcode,g,g_err,g_bibcode)
SELECT main_id, ra, dec, hd, gj,hip,wds,sp_type,sp_bibcode,plx_value,plx_err,plx_bibcode,v,v_err,v_bibcode,g,g_err,g_bibcode
FROM stars_M0to4_nobinaries
'''
c.execute(cmd)
for row in c.fetchall():
    print row

Noticed during Simbad query that one star changed spectral types since last query! Delete the new row (since sp_type ref is much older).

In [24]:
cmd = '''SELECT * FROM stars_FGK_nobinaries 
WHERE main_id LIKE 'CD-46%76' AND sp_type = 'M1V'
'''
c.execute(cmd)
for row in c.fetchall():
    print row

(739, u'CD-46    76', u'5.3317491193017', u'-45.7464190295317', None, u'GJ  9010,GJ    17.1', u'HIP   1696', None, u'M1V', u'1974PhDT........10S', 56.8562, 0.0315, u'2018yCat.1345....0G', 10.394, None, u'2010MNRAS.403.1949K', 9.6286, 0.0004, u'2018yCat.1345....0G')


In [26]:
cmd = '''
DELETE FROM stars_FGK_nobinaries
WHERE main_id LIKE 'CD-46%76' AND sp_type = 'M1V'
'''
c.execute(cmd)

In [27]:
connection.commit()

## Look at the new table

In [31]:
c.execute('PRAGMA table_info(stars_FGK_nobinaries);')
c.fetchall()

[(0, u'sid', u'INTEGER', 0, None, 1),
 (1, u'main_id', u'TEXT', 0, None, 0),
 (2, u'ra', u'TEXT', 0, None, 0),
 (3, u'dec', u'TEXT', 0, None, 0),
 (4, u'hd', u'TEXT', 0, None, 0),
 (5, u'gj', u'TEXT', 0, None, 0),
 (6, u'hip', u'TEXT', 0, None, 0),
 (7, u'wds', u'TEXT', 0, None, 0),
 (8, u'sp_type', u'TEXT', 0, None, 0),
 (9, u'sp_bibcode', u'TEXT', 0, None, 0),
 (10, u'plx_value', u'REAL', 0, None, 0),
 (11, u'plx_err', u'REAL', 0, None, 0),
 (12, u'plx_bibcode', u'TEXT', 0, None, 0),
 (13, u'v', u'REAL', 0, None, 0),
 (14, u'v_err', u'REAL', 0, None, 0),
 (15, u'v_bibcode', u'TEXT', 0, None, 0),
 (16, u'G', u'REAL', 0, None, 0),
 (17, u'G_err', u'REAL', 0, None, 0),
 (18, u'G_bibcode', u'TEXT', 0, None, 0)]

In [32]:
c.execute('SELECT * FROM stars_FGK_nobinaries')
rows = c.fetchall()
print len(rows)
print
for row in rows:
    print row

227

(0, u'* eps Eri', u'53.2326873475', u'-9.45825865694444', u'HD  22049', u'GJ   144', u'HIP  16537', u'WDS J03329-0927A,WDS J03329-0927Aa,Ab', u'K2V', u'1989ApJS...71..245K', 310.94, 0.16, u'2007A&A...474..653V', 3.73, None, u'2002yCat.2237....0D', 3.3691, 0.004, u'2018yCat.1345....0G')
(1, u'* tau Cet', u'26.0170142608333', u'-15.9374795977778', u'HD  10700', u'GJ    71', u'HIP   8102', u'WDS J01441-1556A', u'G8V', u'1989ApJS...71..245K', 273.96, 0.17, u'2007A&A...474..653V', 3.5, None, u'2002yCat.2237....0D', 3.1336, 0.0075, u'2018yCat.1345....0G')
(2, u'* e Eri', u'49.9818788966667', u'-43.0697826375', u'HD  20794', u'GJ   139', u'HIP  15510', None, u'G6V', u'1989ApJS...71..245K', 165.47, 0.19, u'2007A&A...474..653V', 4.27, None, u'2002yCat.2237....0D', 3.9814, 0.0037, u'2018yCat.1345....0G')
(3, u'* del Pav', u'302.18170612375', u'-66.1820675763889', u'HD 190248', u'GJ   780', u'HIP  99240', None, u'G8IV', u'2006AJ....132..161G', 163.71, 0.17, u'2007A&A...474..653V', 3.56, None

## Add columns for R, I, J magnitudes

In [68]:
cmd = '''
SELECT COUNT(*)
FROM stars_FGK_nobinaries
'''
c.execute(cmd)
rows = c.fetchall()
for row in rows:
    print row

(954,)


In [69]:
dat = pandas.read_csv('simbad_FGKM0to4_nobinaries_wRIJmags.csv')
dat.to_sql('stars_RIJmags_temp', connection)

In [11]:
cmd = '''
ALTER TABLE stars_FGK_nobinaries ADD COLUMN r REAL;
'''
c.execute(cmd)
cmd = '''
ALTER TABLE stars_FGK_nobinaries ADD COLUMN r_err REAL;
'''
c.execute(cmd)
cmd = '''
ALTER TABLE stars_FGK_nobinaries ADD COLUMN r_bibcode TEXT;
'''
c.execute(cmd)
cmd = '''
ALTER TABLE stars_FGK_nobinaries ADD COLUMN I REAL;
'''
c.execute(cmd)
cmd = '''
ALTER TABLE stars_FGK_nobinaries ADD COLUMN I_err REAL;
'''
c.execute(cmd)
cmd = '''
ALTER TABLE stars_FGK_nobinaries ADD COLUMN I_bibcode TEXT;
'''
c.execute(cmd)

In [70]:
cmd = '''
ALTER TABLE stars_FGK_nobinaries ADD COLUMN J REAL;
'''
c.execute(cmd)
cmd = '''
ALTER TABLE stars_FGK_nobinaries ADD COLUMN J_err REAL;
'''
c.execute(cmd)
cmd = '''
ALTER TABLE stars_FGK_nobinaries ADD COLUMN J_bibcode TEXT;
'''
c.execute(cmd)

In [71]:
cmd = '''
SELECT J FROM stars_FGK_nobinaries
LIMIT 10
'''
c.execute(cmd)
rows = c.fetchall()
for row in rows:
    print row

(None,)
(None,)
(None,)
(None,)
(None,)
(None,)
(None,)
(None,)
(None,)
(None,)


In [73]:
cmd = ''' 
UPDATE stars_FGK_nobinaries
SET
    r = (SELECT r FROM stars_RIJmags_temp WHERE stars_FGK_nobinaries.main_id = stars_RIJmags_temp.main_id),
    r_err = (SELECT r_err FROM stars_RIJmags_temp WHERE stars_FGK_nobinaries.main_id = stars_RIJmags_temp.main_id),
    r_bibcode = (SELECT r_bibcode FROM stars_RIJmags_temp WHERE stars_FGK_nobinaries.main_id = stars_RIJmags_temp.main_id),
    I = (SELECT I FROM stars_RIJmags_temp WHERE stars_FGK_nobinaries.main_id = stars_RIJmags_temp.main_id),
    I_err = (SELECT I_err FROM stars_RIJmags_temp WHERE stars_FGK_nobinaries.main_id = stars_RIJmags_temp.main_id),
    I_bibcode = (SELECT I_bibcode FROM stars_RIJmags_temp WHERE stars_FGK_nobinaries.main_id = stars_RIJmags_temp.main_id),
    J = (SELECT J FROM stars_RIJmags_temp WHERE stars_FGK_nobinaries.main_id = stars_RIJmags_temp.main_id),
    J_err = (SELECT J_err FROM stars_RIJmags_temp WHERE stars_FGK_nobinaries.main_id = stars_RIJmags_temp.main_id),
    J_bibcode = (SELECT J_bibcode FROM stars_RIJmags_temp WHERE stars_FGK_nobinaries.main_id = stars_RIJmags_temp.main_id)
'''

c.execute(cmd)

rows = c.fetchall()
for row in rows:
    print row

Check new columns:

In [75]:
cmd = '''
SELECT main_id,r,r_err,r_bibcode, J FROM stars_FGK_nobinaries
LIMIT 10
'''
c.execute(cmd)
rows = c.fetchall()
for row in rows:
    print row

(u'* eps Eri', 3.0, None, u'2002yCat.2237....0D', 2.23)
(u'* tau Cet', 2.88, None, u'2002yCat.2237....0D', 2.14)
(u'* e Eri', 3.65, None, u'2002yCat.2237....0D', 2.99)
(u'* del Pav', 2.95, None, u'2002yCat.2237....0D', 2.45)
(u'HD 219134', 4.76, None, u'2002yCat.2237....0D', 3.86)
(u'* bet Hyi', 2.28, None, u'2002yCat.2237....0D', 1.72)
(u'HD 157881', 6.649, None, u'2010MNRAS.403.1949K', 4.934)
(u'HD 217357', 7.013999999999999, None, u'2010MNRAS.403.1949K', 5.346)
(u'*  61 Vir', None, None, None, 3.3339999999999996)
(u'* zet Tuc', 3.73, None, u'2002yCat.2237....0D', 3.19)


In [76]:
cmd = '''
DROP TABLE stars_RIJmags_temp
'''
c.execute(cmd)

## Close connection when done

In [77]:
connection.commit()

In [78]:
connection.close()